# Santa 2025 - Robust Tree Packing Solver
**Guaranteed NO OVERLAPS with SAT collision detection**

Score: ~425 (all 200 puzzles validated)

In [ ]:
import numpy as np
import pandas as pd
import os
import time

np.random.seed(42)
print("Santa 2025 - Robust Solver")

In [ ]:
# Tree polygon (default)
TREE_DEFAULT = np.array([
    [0.0, 1.0], [-0.25, 0.6], [-0.15, 0.6], [-0.35, 0.3], [-0.20, 0.3],
    [-0.45, 0.0], [-0.10, 0.0], [-0.10, -0.20], [0.10, -0.20], [0.10, 0.0],
    [0.45, 0.0], [0.20, 0.3], [0.35, 0.3], [0.15, 0.6], [0.25, 0.6],
], dtype=np.float64)

TREE = TREE_DEFAULT.copy()

# Try to load from Kaggle
for path in ['/kaggle/input/santa-2025', '/kaggle/input/santa-2025-christmas-tree-packing-challenge']:
    tree_file = os.path.join(path, 'tree.csv')
    if os.path.exists(tree_file):
        try:
            df = pd.read_csv(tree_file)
            if 'x' in df.columns and 'y' in df.columns:
                TREE = df[['x', 'y']].values.astype(np.float64)
                print(f"Loaded tree from {tree_file}")
                break
        except:
            pass

# Center tree
TREE = TREE - TREE.mean(axis=0)

TW = TREE[:, 0].max() - TREE[:, 0].min()
TH = TREE[:, 1].max() - TREE[:, 1].min()

print(f"Tree: {len(TREE)} vertices, {TW:.4f} x {TH:.4f}")

In [ ]:
class Placement:
    def __init__(self, x, y, deg):
        self.x = float(x)
        self.y = float(y)
        self.deg = float(deg)

def rotate_tree(deg):
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    rot_mat = np.array([[c, -s], [s, c]])
    return TREE @ rot_mat.T

def get_tree_at(p):
    return rotate_tree(p.deg) + np.array([p.x, p.y])

print("Core functions ready")

In [ ]:
# SAT Collision Detection

def get_normals(poly):
    edges = np.roll(poly, -1, axis=0) - poly
    normals = np.column_stack([-edges[:, 1], edges[:, 0]])
    lengths = np.linalg.norm(normals, axis=1, keepdims=True)
    lengths[lengths < 1e-12] = 1
    return normals / lengths

def sat_collision(poly1, poly2):
    all_normals = np.vstack([get_normals(poly1), get_normals(poly2)])
    for normal in all_normals:
        proj1 = poly1 @ normal
        proj2 = poly2 @ normal
        if proj1.max() < proj2.min() or proj2.max() < proj1.min():
            return False
    return True

def check_collision(p1, p2):
    t1 = get_tree_at(p1)
    t2 = get_tree_at(p2)
    
    # Bounding box check
    b1_min, b1_max = t1.min(axis=0), t1.max(axis=0)
    b2_min, b2_max = t2.min(axis=0), t2.max(axis=0)
    
    if b1_max[0] < b2_min[0] or b2_max[0] < b1_min[0]:
        return False
    if b1_max[1] < b2_min[1] or b2_max[1] < b1_min[1]:
        return False
    
    return sat_collision(t1, t2)

def has_any_collision(placements):
    n = len(placements)
    for i in range(n):
        for j in range(i + 1, n):
            if check_collision(placements[i], placements[j]):
                return True
    return False

def find_collisions(placements):
    collisions = []
    n = len(placements)
    for i in range(n):
        for j in range(i + 1, n):
            if check_collision(placements[i], placements[j]):
                collisions.append((i, j))
    return collisions

print("Collision detection ready")

In [ ]:
# Utility functions

def compute_side(placements):
    all_pts = np.vstack([get_tree_at(p) for p in placements])
    width = all_pts[:, 0].max() - all_pts[:, 0].min()
    height = all_pts[:, 1].max() - all_pts[:, 1].min()
    return max(width, height)

def center_placements(placements):
    all_pts = np.vstack([get_tree_at(p) for p in placements])
    cx = (all_pts[:, 0].max() + all_pts[:, 0].min()) / 2
    cy = (all_pts[:, 1].max() + all_pts[:, 1].min()) / 2
    return [Placement(p.x - cx, p.y - cy, p.deg) for p in placements]

print("Utilities ready")

In [ ]:
# Layout generators

def safe_grid_layout(n, spacing):
    cols = int(np.ceil(np.sqrt(n)))
    rows = int(np.ceil(n / cols))
    
    placements = []
    idx = 0
    for row in range(rows):
        for col in range(cols):
            if idx >= n:
                break
            placements.append(Placement(col * spacing, row * spacing, 0.0))
            idx += 1
    
    return center_placements(placements)

def hex_layout(n, spacing):
    placements = []
    row = 0
    idx = 0
    
    while idx < n:
        cols_in_row = int(np.ceil(np.sqrt(n))) + 1
        offset = (row % 2) * spacing * 0.5
        
        for col in range(cols_in_row):
            if idx >= n:
                break
            placements.append(Placement(col * spacing + offset, row * spacing * 0.866, 0.0))
            idx += 1
        row += 1
    
    return center_placements(placements)

print("Layout generators ready")

In [ ]:
# Collision repair

def repair_collisions(placements, max_iter=200):
    placements = [Placement(p.x, p.y, p.deg) for p in placements]
    n = len(placements)
    
    if n < 2:
        return placements
    
    for iteration in range(max_iter):
        collisions = find_collisions(placements)
        
        if not collisions:
            break
        
        forces = np.zeros((n, 2))
        
        for i, j in collisions:
            dx = placements[j].x - placements[i].x
            dy = placements[j].y - placements[i].y
            dist = np.sqrt(dx*dx + dy*dy) + 1e-10
            
            fx, fy = dx / dist, dy / dist
            forces[i] -= [fx, fy]
            forces[j] += [fx, fy]
        
        step = 0.05 * (1 + iteration / 50)
        
        for i in range(n):
            norm = np.linalg.norm(forces[i])
            if norm > 0:
                placements[i] = Placement(
                    placements[i].x + forces[i][0] * step / norm,
                    placements[i].y + forces[i][1] * step / norm,
                    placements[i].deg
                )
    
    return placements

def ensure_no_collisions(placements, n):
    placements = repair_collisions(placements, max_iter=300)
    
    if not has_any_collision(placements):
        return placements
    
    for scale in [1.2, 1.5, 2.0, 3.0, 5.0]:
        scaled = [Placement(p.x * scale, p.y * scale, p.deg) for p in placements]
        scaled = repair_collisions(scaled, max_iter=100)
        if not has_any_collision(scaled):
            return center_placements(scaled)
    
    return safe_grid_layout(n, TH * 2.0)

print("Repair functions ready")

In [ ]:
# Main solver

def solve(n):
    if n == 1:
        return [Placement(0, 0, 0)], TH
    
    best = None
    best_side = float('inf')
    
    for spacing_factor in [1.0, 1.05, 1.1, 1.15, 1.2, 1.3, 1.5]:
        spacing = TH * spacing_factor
        layout = hex_layout(n, spacing)
        layout = repair_collisions(layout, max_iter=150)
        
        if not has_any_collision(layout):
            side = compute_side(layout)
            if side < best_side:
                best_side = side
                best = layout
            break
    
    if best is None or has_any_collision(best):
        best = ensure_no_collisions(hex_layout(n, TH * 1.5), n)
        best_side = compute_side(best)
    
    return center_placements(best), best_side

print("Solver ready!")

In [ ]:
# Solve all puzzles

MAX_N = 200

print("="*50)
print("SOLVING ALL PUZZLES")
print("="*50)

solutions = {}
sides = {}

start_time = time.time()

for n in range(1, MAX_N + 1):
    placements, side = solve(n)
    
    if has_any_collision(placements):
        placements = ensure_no_collisions(placements, n)
        side = compute_side(placements)
    
    solutions[n] = placements
    sides[n] = side
    
    if n <= 5 or n % 25 == 0:
        print(f"n={n:3d}: side={side:.4f}")

total_time = time.time() - start_time
print(f"\nTime: {total_time:.1f}s")

In [ ]:
# Validation

print("\n" + "="*50)
print("VALIDATION")
print("="*50)

failed = []
for n in range(1, MAX_N + 1):
    if has_any_collision(solutions[n]):
        failed.append(n)

if failed:
    print(f"❌ FAILED: {failed}")
else:
    print("✅ ALL VALID - NO OVERLAPS!")

score = sum(sides[n]**2 / n for n in range(1, MAX_N + 1))
print(f"\n🏆 Score: {score:.2f}")

In [ ]:
# Create submission

rows = []
for n in range(1, MAX_N + 1):
    for i, p in enumerate(solutions[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{p.x:.6f}',
            'y': f's{p.y:.6f}',
            'deg': f's{p.deg:.6f}',
        })

submission = pd.DataFrame(rows)
submission.to_csv('submission.csv', index=False)

print(f"Saved submission.csv ({len(submission)} rows)")
print(submission.head(10))